### 번역자: 고려대학교 수학과(12) 최시현
[AiKorea 번역 프로젝트](https://github.com/aikorea/cs231n)를 이어받아서 진행하였습니다. 기존 AiKorea에서 이미 번역되어있는 CourseNote의 경우 유사한 부분이 많습니다. <br /> 
(깨진 삽화를 추가하거나 개인적으로 번역이 매끄럽지 못하다 생각하는 부분은 수정하였습니다.) <br />
문제가 될시에는 자진 삭제하겠습니다. 오류 수정 및 내용에 대한 기타 문의 사항은 passkmla@naver.com으로 연락주시기 바랍니다.

## Quick intro

뇌에 대한 비유 없이도 신경망을 소개하는 것이 가능하다. <br />
이 장에서는 공식 $s=Wx$,($W$는 행렬, $x$는 이미지의 모든 픽셀 데이터를 담고 있는 입력 열 벡터)를 이용해 <br />
주어진 이미지에 대해 서로 다른 카테고리에 대한 점수를 계산한다. <br />
CIFAR-10의 케이스에서, $x$는 [3072x1]의 열 벡터이고, $W$는 [10x3072] 행렬이다. <br /> 
따라서 결과 스코어는 10개 클래스 스코어에 대한 벡터로 나온다.

뉴럴 네트워크 예제에서는 대신 $s=W_2 max(0, W_1 x)$를 계산할 것이다. <br /> 
여기서 $W_1$은 예를 들면, 이미지를 100차원의 중간 매개 벡터로 바꾸는 [100x3072]행렬이다. <br />
함수 $max(0,-)$는 요소별로 적용되는 비선형 함수이다. 비선형성을 주기 위해서는 여러개의 선택지가 있지만 (추후 공부하게 될 예정), <br /> 
이 함수는 모든 activation에 대하여 0 밑으로는 0이 되는 threshold를 준다. 일반적인 많이 사용된다. <br/ >
마침내, 행렬 $W_2$의 크기는 [10x100]이 되고 따라서 클래스의 스코어로 해석되는 10개의 숫자를 얻게 된다. <br />
비선형성은 계산적으로 매우 중요하다. 만약 이것을 생략한다면, 두 개의 행렬이 하나의 행렬로 합쳐질 수 있고, 따라서 예측된 클래스 스코어는 다시 입력에 대한 선형 함수가 된다. <br />
비선형성은 직관적으로 보면 씰룩 씰룩 움직이는 것이라고 할 수 있다. <br />
파라미터 $W_2$, $W_1$들은 확률적 경사 하강법으로 학습되고, 이들의 그라디언트는 chain-rule로 부터 유도된다. <br />
(backpropagation로부터 계산된다) <br />


3-레이어 신경망은 유사하게 $s=W_3(W_2 max(0, W_1 x))$로 나타낼 수 있고, $W1, W2, W3$는 모두 학습되기 위한 파라미터들이다. <br />
중간 매개 은닉 벡터들의 크기는 네트워크의 하이퍼 파라미터이고, 이것을 설정하는 것은 나중에 살펴 볼 것이다. <br />
어떻게 뉴런/네트워크 관점에서 이 계산들을 해석할 수 있는지 살펴보자.

## 하나의 뉴런 모형화하기

뉴럴 네트워크의 영역은 원래 생물학적 뉴런 시스템을 모델링하는 것을 목표로 영감을 받았다. <br /> 
그러다가 분리되어 머신러닝 과제의 더 나은 결과를 성취하기 위한 엔지니어링 문제가 되었다. <br />
이 분야의 많은 부분이 영향을 받았던 생물학적 시스템의 관점에서 간단하게 논의를 시작해보자.

#### Biological motivation and connections

뇌의 기본 계산 단위는 뉴런이다. 약 8백 60억개의 뉴런들이 인간 신경 체계에서 발견되고, 대략 ${10}^{14}$-${10}^{15}$ 시냅스와 연결되어 있다. <br />
아래의 그림에서는 왼쪽에 보이는 생물학적 뉴런의 삽화와 오른쪽에 일반적인 수치적 모델을 보여준다. <br />
각각의 뉴런은 그들의 수상돌기로부터 입력 신호를 받고, 하나의 출색 돌기를 따라 출력 신호를 만들어 낸다. <br />
이 축색 돌기는 가지를 뻗고 결국에는 다른 뉴런들의 수상돌기와 시냅스로 연결된다. <br />
뉴런의 계산적 모델에서, 신호는 수상 돌기(e.g: $x_0$)를 거쳐 그 시냅스의 강도(e.g: $w_0$)에 기반하여 다른 뉴런의 수상돌기와 <br />
곱해짐으로서(e.g: $w_0 x_0$) 상호작용 하게된다. <br />
이 생각은 시냅스의 강도(가중치 $w$)는 학습가능하고 하나의 뉴런에서 다른뉴런에 강한 영향(Postive Weight)을 주거나, <br /> 
억제(Negative Weight)를 하거나 함으로서 조절할 수 있다. <br />
이 기본 모형에서, 수상 돌기는 신호를 신체 세포로 옮기고 그곳에서 모두 더해진다. <br />
만약 최종 합이 특정한 임계치를 넘어서면, 뉴런은 활성화되고 이 Spike는 그것의 축색 돌기를 따라 보내진다. <br />
계산적 모형에서는 Spike의 정확한 타이밍은 중요하지 않고 오직 정보와 firing comminicates information의 빈도가 중요하다고 가정한다. <br />
축색 돌기를 따라 얼마나 자주 Spike하는지를 나타내는 활성화 함수 $f$와 뉴런의 fire rate를 모델링한다. <br />
역사적으로, 활성화 함수는 일반적으로 시그모이드 함수 $\sigma$를 사용한다. 왜냐하면, 이 함수는 실제 입력 값을(합쳐진 다음의 신호 강도) 받아서 <br />
0과1 사이의 범위로 조정하기 때문이다. 이 장 이후에 이들 활성화 함수에 대해서 상세히 살펴볼 것이다.
___

<img src="./Images/1.png" width=600 />

생물학적 뉴런의 삽화적 표현(왼쪽)과 수리적 모형(오른쪽)
___

단일 뉴런이 forward-propagating을 하는 예제는 다음과 같다.

In [ ]:
class Neuron(object):
  # ... 
  def forward(self, inputs):
    """ assume inputs and weights are 1-D numpy arrays and bias is a number """
    cell_body_sum = np.sum(inputs * self.weights) + self.bias
    firing_rate = 1.0 / (1.0 + math.exp(-cell_body_sum)) # sigmoid activation function
    return firing_rate

즉, 각 뉴런들은 입력과 그것의 가중치들에 대하여 내적 연산을 수행하고, bias를 더하고, 비선형 함수(활성화 함수)를 적용한다. <br />
이 경우에는 시그모이드 $\sigma (x)=\frac { 1 }{ 1+{ e }^{ -x } } $를 활성화 함수로 사용한다. <br />
이 절의 끝에서 다른 활성화 함수에 대해서 자세히 알아볼 것이다.

#### Coarse 모델

생물학전 뉴런의 모형이 매우 coarse하다는 것은 매우 중요하다. <br />
예를 들어, 각각의 다른 성질을 가지고 있는 매우 다양한 종류의 뉴런들이 존재한다. <br />
생물학적 관점에서 수상돌기는 매우 복잡한 비선형적 계산을 수행한다. <br />
시냅스는 단지 단순한 하나의 가중치가 아니라 복잡한 비선형 동역학 체계이다. <br />
출력 Spike의 정확한 타이밍은 많은 시스템에서 중요하다고 알려졌고, rate code approximation은 그렇지 않다고 제안한다. <br />
이들과 많은 다른 단순함 때문에, 뉴럴 네트워크과 실제 뇌의 유사성에 대하여 이야기를 하고자 한다면, <br /> 
몇몇 신경 과학 배경을 가진 누군가로부터 그르렁 거리는 소리를 들을 준비를 해야할 것이다. <br />
 See this [review](https://neurophysics.ucsd.edu/courses/physics_171/annurev.neuro.28.061604.135703.pdf) (pdf), or more recently this [review](http://www.sciencedirect.com/science/article/pii/S0959438814000130) if you are interested.

## 선형 분류기로써 단일 뉴런

뉴런 모형의 forward 계산의 수리적 형식는 우리에게 친숙하다. <br />
선형 분류기에서 본 것 처럼, 뉴런은 입력의 특정 선형 영역에서 "좋다"(활성이 1에 가까움), "싫다"(활성이 0에 가까움)를 구분하는 능력을 가진다. <br />
따라서 뉴런의 출력에 대한 적절한 손실 함수와 함께 단일 뉴런을 선형 분류기로 바꿀 수 있다. <br />

#### 이항 소프트 맥스 분류기

예를 들어, $\sigma (\sum _{ i }^{  }{ { w }_{ i }{ x }_{ i }+b } )$를 한 클래스의 확률 $P({ y }_{ i }=1|{ x }_{ i };w)$로 해석할 수 있다. <br />
다른 클래스의 확률은 $P({ y }_{ i }=0|{ x }_{ i };w)=1-P({ y }_{ i }=1|{ x }_{ i };w)$이 될것이다. <br />
따라서 이 둘의 합은 1이 된다. 이런 해석을 통해 선형 분류기 섹션에서 본 것 처럼 크로스 엔트로피를 formulate 할 수 있고, <br />
로지스틱 회귀로 알려진 이항 소프트맥스 분류기를 따르도록 최적화 할 수 있다. 시그모이드 함수가 0과 1사이로 제한되어 있기 때문에, <br />
이 분류기에 대한 예측은 뉴런의 출력이 0.5보다 큰지 아닌지에 달려있다.

#### 이항 SVM 분류기

대안적으로, 최대 마진 hinge loss를 뉴런의 출력에 적용할 수 있고, 이것을 학습시켜 이항 SVM이 되게 할 수 있다.

#### 정규화(regularization) 해석

SVM이나 소프트맥스의 사례에서 정규화(regularization) 손실은 생물학적 관점에서 점진적 망각으로 해석할 수 있다. <br />
왜냐하면 이것은 모든 파라미터를 업데이트 한 후 시냅틱 가중치 $w$를 모두 0으로 유도하는 효과를 가지고 있기 때문이다.

> 단일 뉴런은 이항 분류기를 구현하는것처럼 사용될 수 있다.(e.g: 이항 소프트맥스 또는 이항 SVM 분류기)

#### 일반적으로 사용되는 활성화 함수들

모든 활성화 함수(비선형성는 하나의 숫자를 가지고 특정 fixed된 수리적 연산을 수행한다. <br />
실전에서 만날 수 있는 몇가지 활성화 함수들이 있다.
___

<img src="./Images/2.png" width=600 />

왼쪽: 시그모이드 비선형성은 실수들을 [0, 1]사이로 조정한다. <br />
오른쪽: tanh 비선형은 실수들을 [-1, 1]사이로 조정한다. 
___

#### 시그모이드

시그모이드 비선형성은 수리적 형식 (위 그림 왼쪽에 표현됨) $\sigma (x)=\frac { 1 }{ 1+{ e }^{ -x } } $를 갖는다. <br />
이전 절에서 암시한 것처럼, 이것은 실수들을 0과 1사이로 조정한다. 특히, 매우 큰 음수는 0이 되고, 매우 큰 양수는 1이 된다. <br />
이 시그모이드 함수는 firing rate of a neuron로 잘 해석할되므로 역사적으로 자주 사용되어 왔다. <br/ >
0에서는 not firing at all, 최대 빈도 1에서 fully-saturated firing 한다고 가정한다. <br />
실제로 시그모이드 비선형성은 최근에는 잘 사용하지 않고 간간히 사용된다. 이것은 두 가지 주요 문제점이 있다.

- 시그모이드는 포화가 되고나면 그라디언트를 죽인다. <br /> 
시그모이드 뉴런의 바람직하지 않은 특성은 뉴런의 활성화 때 0이나 1의 꼬리에서 포화가 된다는 것이다. <br />
이 영역에서 그라디언트는 거의 0이다. <br /> 
backpropagation일 때를 생각해보면, 전체 목적함수(objective)에서 이 지역적 그라디언트를 이 게이트의 출력의 그라디언트에 곱하게 된다. <br />
그러므로, 만약 지역 그라디언트가 매우 작다면, 이것은 효과적으로 그라디언트를 죽인다. <br /> 
즉, 뉴런을 따라서는 이것의 가중치들과 재귀적으로 데이터에서 거의 아무런 신호도 흐르지 않을 것이다. <br />
따라서, 포화를 막기 위해서는 시그모이드 뉴런의 가중치들을 초기화 할 때 특히 주의를 해야한다. <br />
예를 들어, 만약 초기 가중치가 너무 크면 대부분의 뉴런들은 포화될 것이고 네트워크는 거의 학습하지 못한다.

- 시그모이드 출력은 0이 중심으로 하지 않는다. <br />
이것은 바람직하지 않다. <br /> 
왜냐하면, 뉴럴 네트워크의 프로세스에서 layer 레이어의 뉴런은 0이 중심이 되지 않은 데이터를 받게 되기 때문이다. <br />
이는 그라디언트 디센트 동안 다이나믹한 영향을 가진다. <br /> 
만약 항상 뉴런에 데이터가 postive로 들어오게 되면(e.g: $f={ w }^{ T }x+b$에서 원소마다 $x>0$), <br /> 
backpropagation 동안 가중치 $w$에 대한 그라디언트는 전부 양이거나 전부 음이 될 수 있기 때문이다. <br /> 
(전체 표현식 $f$의 그라디언트에 달려있다) <br />
이것은 가중치들에 대한 그라디언트 업데이트에서 바람직하지 않은 zig-zagging 다이나믹이 될 수 있다. <br />
그러나 일단 이 그라디언트들을 batch of data를 따라 더한 다음 최종 가중치들을 업데이트한다면 <br /> 
다양한 부호를 가질수 있고 다소 이 문제를 완화할 수 있다. <br />
이는 불편하지만, 위의 activation 포화문제에 비하면 덜 심각한 수준이다. <br />

#### Tanh

Tanh 의 비선형성은 위의 오른쪽 그림과 같다. <br />
이것은 실수를 [-1, 1]사이로 조정한다. 시그모이드 뉴런처럼 이 함수 역시 포화되지만, 시그모이드 뉴런과는 다르게 결과가 0을 중심으로 되어있다. <br />
그러므로 실제로 tanh 비선형성은 시그모이드 비선형성보다 항상 더 선호된다. <br />
또한 tanh는 시그모이드 뉴런을 간단히 scaled한 것이다. 특히 다음과 같은 식이 성립한다. $tanh(x)=2\sigma (2x)-1$
___

<img src="./Images/3.png" width=600 />

왼쪽: ReLU activation 함수, x가 0보다 작으면 0이고, x가 0보다 크면 기울기가 1을 갖는다. <br />
오른쪽: Tanh와 비교하여 수렴에 대하여 보통 6배의 성능 향상이 있다고 한다.
___

#### ReLU

ReLU는 최근 몇 년 사이에 굉장히 유명해졌다. 이것은 함수 $f(x)=max(0,x)$를 계산한다. <br />
즉, 이 activation은 0에서 threshold를 갖는다.(위의 왼쪽 그림) <br />
ReLU를 사용하는 것에 대해 몇가지 장단점이 있다.

- (+) 이것은 시그모이드나 tanh 함수와 비교하여 확률적 기울기 하강의 수렴에서 굉장히 빠른 가속이 있다. <br />
이것은 선형성과 포화되지 않음 때문이다. (e.g. a factor of 6 in [Krizhevsky et al.](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf)) 
- (+) Cost가 높은 연산을 포함하고 있는 시그모이드와 tanh에 비교하여 이것은 매우 간단하게 0에서 activation되는 threshold를 갖는 행렬로 구현될 수 있다.
- 안타깝게도, ReLU 유닛은 학습하는동안 취약할 수 있고 쉽게 죽을 수 있다. <br />
예를 들어, ReLU 뉴런을 따라 흐르는 큰 그라디언트는 가중치가 업데이트되는 과정에서 <br /> 
그 뉴런이 어떤 datapoint에서도 다시는 activation되지 않을 수도 있다. <br />
이것이 일어나면 그 유닛을 따라 흐르는 그라디언트sms 그 점에서 영원히 0이 될 수 있다. <br />
즉, ReLU 유닛이 학습되는 동안 되돌릴수 없이 죽게된다. 왜냐하면 그들은 data manifold에서 벗어날 수 있기 때문이다. <br />
예를 들어, 학습 속도가 너무 높다면 40%정도의 네트워크가 죽을 수 있다. <br />
(i.e: 이 뉴런들은 전체 데이터 셋에 대하여 절대 활성화 되지 않는다) <br />
적절한 학습 속도의 설정과 함께, 이 문제가 완화될 수 있다. <br />

#### Leak ReLU

Leak ReLU는 ReLU가 죽는 문제를 해결하기 위한 하나의 시도이다. <br />
함수가 x보다 작을 때 0이 되는 것 대신에, Leak ReLU는 아주 작은 음의 기울기(0.01 등등...)을 갖는다. <br /> 
즉, 함수는 $f(x)=1(x<0)(\alpha x)+1(x>=0)(x)$ 여기서 $\alpha$는 매우 작은 상수이다. <br />
몇몇 사람들은 activation 함수의 이 형태가 성공적이라고 보고하지만, 그러나 결과가 항상 일관되지는 않는다. <br />
The slope in the negative region can also be made into a parameter of each neuron, <br /> 
as seen in PReLU neurons, introduced in [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), by Kaiming He et al., 2015. <br />
그러나 task에 대한 일관적인 benifit은 여전히 불명확하다. <br />

#### Maxout

가중치와 데이터와의 내적으로 적용된 함수적 형태 $f({ w }^{ T }x+b)$의 비선형성이 가지고 있지 않은 다른 유형의 유닛이 제안되고 있다. <br />
상대적으로 유명한 선택이 ReLU와 leaky 버전을 일반화한 Maxout 뉴런이다. <br />
Maxout 뉴런(introduced recently by [Goodfellow et al.](http://www-etud.iro.umontreal.ca/~goodfeli/maxout.html))은 함수 max(${ w }_{1}^{ T }x+{b}_{1}$, ${ w }_{2}^{ T }x+{b}_{2}$)이다. <br />
ReLU와 Leaky ReLU는 이 형태의 특별한 사례이다.(예를 들어, ReLU는 $w_1, b_1=0$이다) <br />
그러므로 Maxout 뉴런은 ReLU의 장점(포화되지 않고, 연산의 선형 체제)은 가지고 이들의 문제점(ReLU가 죽는 것)은 갖지 않는다. <br />
그러나, ReLU 뉴런과 다르게 이것은 모든 단일 뉴런에 대해 파라미터들의 숫자를 두배로 만들고, 전체 파라미터의 갯수가 매우 많아지게 된다.

이것으로 activation 함수와 뉴런의 일반적인 형식에 대한 논의를 끝냈다. <br /> 
마지막으로, 이것을 함으로 인해 근본적인 문제가 없더라도
같은 네트워크에서 다른 종류의 뉴런들을 섞어서 매치하는 것은 드물다.

#### TLDR

"내가 사용해야 하는 뉴런의 형태는 무엇입니까?", ReLU 비선형성을 사용해라. <br /> 
네트워크에서 "죽는" 유닛의 비율을 가능한 관찰하고, 학습률을 조심해라. <br />
만약 이것이 염려된다면, Leak ReLU나 Maxout을 시도해라. 절대로 시그모이드는 사용하지 마라. 차라리 tanh를 사용해라. <br />
하지만 이것은 ReLU나 Maxout보다 더 좋지 않게 작동할 것이다,

## 뉴럴 네트워크 아키텍쳐

### Layer-wise organization

#### 그래프에서 뉴런으로서의 뉴럴 네트워크

뉴럴 네트워크는 비순환적인 그래프에서 연결된 뉴런들의 집합으로 모델링된다. <br/ >
즉, 몇몇 뉴런의 출력은 다른 뉴런의 입력이 된다. 순환은 허용되지 않는다. 왜냐하면 그것은 네트워크의 forward pass에 대하여 무한 순환을 의미하기 때문이다. <br />
연결된 뉴런의 형태가 없는 blobs 대신에, 뉴런 네트워크 모델들은 종종 구별되는 뉴런의 층으로 조직된다. <br />
정규(regular) 뉴럴 네트워크에 대하여, 가장 일반적인 레이어 형태는 Fully-connected-레이어이고 이것은 인접한 두 레이어 사이에 서로 완전히 쌍대 연결된 레이어이다. <br />
하지만 뉴런들은 한 레이어 내에서는 연결되지 않는다. 아래는 Fully-connected-레이어들을 쌓은 두 뉴럴 네트워크 위상의 예제가 있다.
___

<img src="./Images/4.png" width=700 />

왼쪽: 2-레이어 신경망(하나의 은닉 레이어는 4개의 뉴런을 가지고 있고, 하나의 출력 레이어는 2개의 뉴런을 가지고 있다)과 세 개의 입력 <br />
오른쪽: 3개의 입력을 갖는 3-레이어 신경망, 2개의 은닉 레이어는 각각 4개의 뉴런을 갖고, 하나의 출력 레이어를 갖는다. <br />
두 사례 모두 레이어들 사이의 뉴런들은 연결되어 있지만, 한 레이어 내에서는 연결되지 않는다. 
___

#### 이름 지정 규칙

N-레이어 뉴럴 네트워크를 말할 때, 입력층을 세지 않는다. 그러므로 단일 레이어 뉴럴 네트워크는 히든 레이어가 없이 입력에 바로 출력에 직접적으로 매핑되는 망을 말한다. <br />
그런 점에서, 때때로, 로지스틱 회귀나 SVM이 단순히 단일 레이어 신경망의 특별한 매핑 형태라고 말하는 것을 들을 수 있다. <br />
또한 이런 네트워크들이 "인공 뉴럴 네트워크", "다층 퍼셉트론(MLP)"라고 바꾸어 부르는 것을 들을 수 있다. <br />
많은 사람들이 신경망과 실제 뇌 사이의 비유를 좋아하지 않고, 유닛을 뉴런으로 부르는 것을 선호한다.

#### 출력 레이어

뉴럴 네트워크에서의 모든 레이어와 다르게, 출력 레이어 뉴런들은 대부분 공통적으로 activation 함수를 가지지 않는다.(즉, 이것들이 선형 동일 activation 함수를 갖는다고 생각할 수 있다)
이것은 왜냐하면 마지막 출력 레이어가 대게 임의적인 실수값들로 클래스 스코어(e.g:분류)나 실제 목표값의 종류(e.g: 회귀)를 표현하기 때문이다. <br />

#### 신경망의 크기 조정하기

사람들이 일반적으로 뉴럴 네트워크의 크기를 측정하는데 두 가지 방법을 사용한다. <br /> 
하나는 뉴런의 숫자이고 더 일반적으로 사용되는 것은 파라미터들의 수이다. <br />
위 그림에서 두 예제 네트워크를 통해 살펴보자.
- 첫 번째 네트워크(왼쪽)은 4+2=6의 뉴런을 가지고 있고(입력은 세지 않는다) [3x4]+[4x2]=20개의 가중치들과 4+2=6의 bias, 총 26개의 학습가능한 파라미터들을 가지고 있다. <br />
- 두 번째 네트워크는 4+4+1=9개의 뉴런을 가지고 있고, [3x4]+[4x4]+[4x1]=12+16+4=32개의 가중치들과 4+4+1=9의 bias, 총 41개의 학습가능한 파라미터들을 가지고 있다.

몇가지 배경을 이야기하자면, 최근 ConvNet들은 1억개의 파라미터들을 갖고, 대개 약 10-20개의 층으로 구성된다.(따라서 딥러닝이라 부른다) <br />
그러나 앞으로 살펴보게 되겠지만, 효율적인 연결의 숫자는 파라미터 공유 때문에 더 중요해 진다. <br />
더 많은 내용은 ConvNet 모듈에서 살펴보자.

## Feed-forward 계산 예제

#### activation 함수를 갖는 반복적인 행렬 곱셈

뉴럴 네트워크들이 층별로 조직되는 가장 중요한 이유는 이 구조는 행렬의 벡터 연산을 사용하여 뉴럴 네트워크를 평가하는데 매우 간단하고 효율적이기 때문이다. <br />
위의 다이어그램에서 본 3-레이어 뉴럴 네트워크의 예제에서, 입력은 [3x1]벡터이다. 레이어에 대한 모든 연결 강도는 단일 행렬에 저장되어있다. <br />
예를 들어, 첫 번째 히든 레이어의 가중치는 [4x3]크기의 W1이고, 모든 유닛에 대한 bias는 벡터 b1이다. <br />
여기서, 모든 단일 뉴런은 W1의 행에 그것의 가중치들을 갖는다. <br />
따라서 행렬 벡터의 곱 np.dot(W1,x)는 그 레이어의 모든 뉴런들의 activation을 평가한다. <br />
유사하게, W2는 2번째 히든 레이어를 연결을 저장하고 있는 [4x4]행렬이고, W3는 마지막 출력 레이어에 대한 [1x4]행렬이다. <br />
이 3-레이어 뉴럴 네트워크의 full forward pass는 activation 함수와 함께 엮어만든 간단한 세 행렬의 곱이다. 

In [ ]:
# forward-pass of a 3-layer neural network:
f = lambda x: 1.0/(1.0 + np.exp(-x)) # activation function (use sigmoid)
x = np.random.randn(3, 1) # random input vector of three numbers (3x1)
h1 = f(np.dot(W1, x) + b1) # calculate first hidden layer activations (4x1)
h2 = f(np.dot(W2, h1) + b2) # calculate second hidden layer activations (4x1)
out = np.dot(W3, h2) + b3 # output neuron (1x1)

위의 코드에서, W1, W2, W3, b1, b2, b3는 네트워크의 학습 가능한 파라미터이다. <br />
또한 단일 입력 열 벡터를 가지는 대신에, 변수 x는 학습 데이터의 전체 batch를 다 가지고 있고(각각의 입력 예제는 x의 행이다) <br />
모든 예제는 효과적으로 병렬적으로 계산된다. <br />
마지막 뉴럴 네트워크 레이어는 보통 activation 함수를 갖지 않는다는 것을 명심해라.(e.g: 이것은 분류의 설정에서 클래스 스코어(실제 값)을 나타낸다)

> Fully-connected 층의 forward pass는 bias offset과 activation 함수가 있는 하나의 행렬 곱에 대응한다.

### 표현 능력

Fully-connected layers를 갖는 뉴럴 네트워크를 보기 위한 한 가지 방법은 그들을 네트워크의 파라미터화된 가중치들의 함수의 집합으로 정의하는 것이다. <br />
자연적으로 생기는 질문은, 함수들의 집합의 표현 능력은 무엇인가?, 특히 뉴럴 네트워크와 함께 모델링 될 수 없는 함수들이 있는가? <br />

적어도 하나의 히든 레이어가 Universal approximators함이 밝혀졌다. <br />
(e.g. see [Approximation by Superpositions of Sigmoidal Function](http://www.dartmouth.edu/~gvc/Cybenko_MCSS.pdf) from 1989 (pdf), or this [intuitive explanation](http://neuralnetworksanddeeplearning.com/chap4.html) from Michael Nielsen)
즉 이것은 어떤 연속 함수 $f(x)$와 $\varepsilon >0$가 주어지면, $\forall x,|f(x)-g(x)|<\varepsilon$을 만족하는 하나의 은닉층 <br /> 
(비선형을 택하는 합리적인 선택과 함께, e.g: 시그모이드)을 갖는 신경망 $g(x)$가 존재한다. <br /> 
즉, 이 신경망은 연속적인 함수에 근사할 수 있다.

만약, 하나의 은닉층이 어떤 함수를 근사하는데 충분하다면, 왜 더 많은 레이어와 더 깊은층을 가지려하는가? <br />
이 답은 실제 2-레이어 신경망에서 universal approximators는 수학적으로 쉽지만, 상대적으로 약하고 쓸모없는 상태이기 때문이다. <br />
일차원에서, "sum of indicator bumps" 함수 $g(x)=\sum _{ i }^{  }{ { c }_{ i } } 1({ a }_{ i }<x<{ b }_{ i })$(a,b,c는 파라미터 벡터)는 universal approximators이다. <br />
그러나 그 누구도 머신러닝에서 이 함수의 형식을 사용하라고 제안하지 않는다. <br />
뉴럴 네트워크는 실제로 잘 작동한다. <br /> 
왜냐하면 그들은 실제에서 마주치는 자료들의 통계적 특성과 잘 맞아떨어지게 작동하는 멋지고 smooth한 함수들이기 때문이다. <br />
그리고 최적화 알고리즘 (e/g: 그라디언트 디센트)를 사용하여 학습하기 쉽다. <br />
유사하게, 딥 네트워크(다수의 히든 레이어)는 그들의 능력적 표현이 동등함에도 불구하고 <br /> 
실험적 관찰에서 보았을 때 단일 레이어보다 더 잘 작동한다. 

예외로, 실제에서 자주 발견되는 케이스는 3-레이어 신경망이 2-레이어 신경망보다 더 뛰어나다는 것이다. <br />
그러나 더 깊어진다고(4, 5, 6 레이어) 하더라도 더 도움이 되지는 않는다. <br />
하지만 이것은 ConvNet에서는 대조적으로, 깊이는 좋은 인지 체계를 위해 매우 중요한 요소임이 밝혀졌다. <br />
(e.g: 10개의 학습 가능한 레이어의 순서) 이것에 대한 한가지 주장은 이미지는 계층적 구조를 갖는다는 것이다. <br />
(e.g: 얼굴은 눈으로 구성되고 눈은 모서리로 구성되고 등등...)<br />
그래서 프로세스의 여러 레이어들은 이 자료 영역에 대한 직관적인 감각을 만든다.

The full story is, of course, much more involved and a topic of much recent research. If you are interested in these topics we recommend for further reading:

- [Deep Learning](http://www.deeplearningbook.org/) book in press by Bengio, Goodfellow, Courville, in particular [Chapter 6.4.](http://www.deeplearningbook.org/contents/mlp.html)
- [Do Deep Nets Really Need to be Deep?](https://arxiv.org/abs/1312.6184)
- [FitNets: Hints for Thin Deep Nets](https://arxiv.org/abs/1412.6550)

#### 레이어의 수와 크기 결정하기

어떻게 실제 문제에 직면했을 때 유용한 아키텍처가 무엇인지 결정할 수 있을까? <br />
은닉 레이어를 사용하지 않아야 할까? 하나의 은닉 레이어? 두개의 은닉 레이어? ? 각 레이어는 얼마나 커야 할까? <br />
첫째는, 뉴럴 네트워크에서 레이어의 수와 크기를 증가시킬수록, 네트워크의 수용량은 증가한다. <br />
즉, 표현가능한 함수의 공간은 증가한다. 왜냐하면 뉴런은 다른 많은 함수들과 함께 협업하여 표현할 수 있기 때문이다. <br />
예를 들어, 2차원에 대하여 이항 분류기 문제를 가지고 있다고 하자. 3개의 다른 뉴럴 네트워크를 학습시킬 수 있고, <br /> 
각각 어떤 크기의 히든 레이어와 함께 다음의 분류기를 얻을 수 있다.
___

<img src="./Images/5.png" width=700 />

더 큰 뉴럴 네트워크는 더 복잡한 함수들을 표현할 수 있다. <br />
데이터를 그들의 클래스에 따라 색칠된 점으로 나타내었고, 네트워크에 의해 학습된 decision regions를 보여준다. <br />
You can play with these examples in this [ConvNetsJS demo.](http://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html)
___

위의 그림처럼, 네트워크에서 더 많은 뉴런들이 더 복잡한 함수를 표현할 수 있음을 보았다. <br />
그러나 이것은 장단점이 존재한다. 장점(더 복잡한 데이터 분류를 위한 학습을 할 수 있는 것), 단점(확습 자료에 쉽게 과적합 됨) <br />
과적합은 높은 capacity를 갖는 모델이 학습 데이터의 노이즈에게도 적합할 때 발생한다. <br />
예를 들어, 20개의 히든 뉴런을 갖는 모델은 모든 학습 데이터에 적합하지만 <br /> 
공간을 분리된 여러개의 빨간 지역과 녹색 지역의 dicision regions로 나누는데에 큰 비용이 든다. <br />
3개의 히든 뉴런을 가지는 모델은 오직 넓은 줄기를 분류하는 힘만을 표현한다. <br />
이것은 데이터를 두개의 색깔로 모델링하고, 몇몇의 작은 수의 빨간 점들을 녹색 군집안에 아웃라이어(노이즈)로서 포함시키도록 해석한다. <br />
실제로, 이것은 테스트 집합에 대해 더 나은 일반화를 이끌어낸다.

위에서 논의한 것에 기초하여 생각하면, 만약 데이터가 과적합을 일으킬 정도로 너무 복잡하지 않다면 <br /> 
더 작은 뉴럴 네트워크가 선호될 수 있는 것 처럼 보인다. 하지만 이것은 잘못된 생각이다. <br /> 
나중에 살펴볼 것이지만 뉴럴 네트워크에서 과적합을 막을 수 있는 다양한 다른 더 좋은 방법들이 있다. <br />
실제로 뉴런의 수를 통제하는 대신에 과적합을 위한 통제 방법이 더 낫다.

이 뒤에 숨겨진 미묘한 이유는 더 작은 네트워크들은 그라디언트 디센트와 같은 지역적인 학습을 할 때 더 어렵다. <br />
이들의 손실 함수가 상대적으로 더 적은 지역 최소화를 가지지만, 이 최소값들중 많은 값들로 더 쉽게 수렴을 한다. <br /> 
그리고 이들은 매우 나쁘다. (i.e: 높은 손실 값으로 수렴한다) <br />
반대로, 더 큰 뉴럴 네트워크는 훨씬 더 많은 지역 최소점은 포함하고, 이 최소점들은 그들의 실제 손실의 측면에서 훨씬 더 낫다는 것이 밝혀졌다. <br />
뉴럴 네트워크들은 볼록하지 않기 때문에, 이런 성질들을 수학적으로 밝히는 것이 어렵다는 것이 밝혀졌지만, <br /> 
이들의 목적 함수(objective)가 만들어내는 것을 시도하기 위한 몇가지 시도들이 있다. <br />
e.g. in a recent paper [The Loss Surfaces of Multilayer Networks.](https://arxiv.org/abs/1412.0233) <br />
실제로 만약 작은 네트워크를 학습한다면, 최종 손실은 적당한 크기의 분산으로 나타날 것이다. <br />
몇몇 케이스에서 운이 좋으면 좋은 값에 수렴을 할 수도 있지만, 몇몇 케이스에서는 나쁜 최소값에 갇힐 수도 있다. <br />
반면에, 큰 네트워크를 학습시키려고 한다면, 많은 다른 해법들을 찾기 시작할 것이다. <br /> 
그러나 최종으로 얻어진 손실의 분산은 더 작을 것이다. 즉, 모든 해들은 거의 동등하게 좋지만, 무작위 초기화의 운에는 덜 기대야 한다.

다시 말해, 정규화(regularization) 강도는 뉴럴 네트워크의 과적합을 통제하기 위해 선호되는 방법이다. <br />
세가지 다른 설정에 대해 얻어지는 결과를 다음 그림을 통해 볼 수 있다.

___

<img src="./Images/7.png" width=700 />

<img src="./Images/6.png" width=700 />

정규화(regularization) 강도의 효과 <br />
각각의 뉴럴 네트워크는 위에서처럼 20개의 은닉 뉴런을 가지고 있지만, <br />
정규화(regularization) 강도를 변화시키는 것은 이들의 최종 decision regions를 더 높은 정규화(regularization)를 가지고 부드럽게 만든다. <br />
You can play with these examples in this [ConvNetsJS demo.](http://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html)
___

과적합의 위험성이 있기 때문에 작은 네트워크를 사용한다는 생각은 버리는 것이 좋다. <br />
대신에, 우리의 계산적 자원이 허용되는 범위에서 최대한 큰 신경망을 만들어야 한다. <br />
과적화을 막기 위해 정규화(regularization) 기법들을 사용해라.

## Summary

In summary,

- We introduced a very coarse model of a biological neuron
- We discussed several types of activation functions that are used in practice, with ReLU being the most common choice
- We introduced Neural Networks where neurons are connected with Fully-Connected layers where neurons in adjacent layers have full pair-wise connections, but neurons within a layer are not connected.
- We saw that this layered architecture enables very efficient evaluation of Neural Networks based on matrix multiplications interwoven with the application of the activation function.
- We saw that that Neural Networks are universal function approximators, but we also discussed the fact that this property has little to do with their ubiquitous use. They are used because they make certain “right” assumptions about the functional forms of functions that come up in practice.
- We discussed the fact that larger networks will always work better than smaller networks, but their higher model capacity must be appropriately addressed with stronger regularization (such as higher weight decay), or they might overfit. We will see more forms of regularization (especially dropout) in later sections.

## Additional References

- [deeplearning.net tutorial](http://www.deeplearning.net/tutorial/mlp.html) with Theano
- [ConvNetJS](http://cs.stanford.edu/people/karpathy/convnetjs/) demos for intuitions
- [Michael Nielsen’s](http://neuralnetworksanddeeplearning.com/chap1.html) tutorials